# **과제1 : 챗봇만들기**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기1
    * Vector DB 준비하기
    * Retriever, LLM를 연결하기

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key[:5])

sk-pr


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩

In [3]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data.head()

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : ./db



In [4]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

database = Chroma(persist_directory='./db', embedding_function=embeddings)

* 데이터 입력
    * DF to Vector DB

In [5]:
text_list = data['QA'].to_list()

documents = [Document(page_content=text) for text in text_list]

print(documents)

[Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'), Document(page_content='35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)'), Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'), Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.'), Document(page_content='아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,\n고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기

* 입력된 데이터 조회

In [6]:
database.add_documents(documents)

['5e58e870-acad-4973-9e40-0702e5c65f4b',
 'a16c1b19-77f4-4559-ae66-58ff21da2fe2',
 '5a74318d-926c-4e01-9753-424b9f88f585',
 '2db569fb-4129-4b43-998e-215102620cb3',
 '0aada355-d5b0-4c5c-a0b1-0718c3eb33fa',
 '19ab1821-ff0a-4800-83cd-0043fc4c50e4',
 '82dff4ab-bd9e-4742-ae05-30dc5fc7dc4b',
 'd105ef27-3198-45e3-b5d0-a94b344fcdc1',
 '19ec4a0c-940b-47e2-9479-3ca59b3debf5',
 '31d50512-24d6-42f6-b524-cd2d9d6e8ac1']

In [7]:
database.get()

{'ids': ['0aada355-d5b0-4c5c-a0b1-0718c3eb33fa',
  '19ab1821-ff0a-4800-83cd-0043fc4c50e4',
  '19ec4a0c-940b-47e2-9479-3ca59b3debf5',
  '2db569fb-4129-4b43-998e-215102620cb3',
  '31d50512-24d6-42f6-b524-cd2d9d6e8ac1',
  '5a74318d-926c-4e01-9753-424b9f88f585',
  '5e58e870-acad-4973-9e40-0702e5c65f4b',
  '82dff4ab-bd9e-4742-ae05-30dc5fc7dc4b',
  'a16c1b19-77f4-4559-ae66-58ff21da2fe2',
  'd105ef27-3198-45e3-b5d0-a94b344fcdc1'],
 'embeddings': None,
 'metadatas': [None, None, None, None, None, None, None, None, None, None],
 'documents': ['아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,\n고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기 바랍니다.\n※ 수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 

## **3.RAG+LLM모델**

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

In [35]:
k = 5
query = '에이블 스쿨 지원 시 제한되는 조건이 있나요?'

sim_documents = database.similarity_search_with_score(query, k = k)
for doc in sim_documents:
    print(f'유사도 점수 : {round(doc[1], 5)}, 문서 내용 : {doc[0].page_content}')

유사도 점수 : 0.28549, 문서 내용 : 직장인도 지원할 수 있나요?
KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.
유사도 점수 : 0.32997, 문서 내용 : 최종 학력 또는 전공과 관계없이 지원할 수 있나요?
KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.
유사도 점수 : 0.35348, 문서 내용 : 35세 이상은 지원할 수 없나요?
본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)
유사도 점수 : 0.36135, 문서 내용 : 현재 군 복무중인 상태인데, 지원이 가능한가요?
현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.
유사도 점수 : 0.36637, 문서 내용 : 다른 K-DT 과정을 들었는데 지원할 수 있나요?
K-DT 교육과정은 최초 1회에 한해서만 무료 수강이 가능하며, KT 에이블스쿨은 본인 부담금이 발생하는 경우 지원이 어렵습니다. K-DT 과정마다 차이가 있을 수 있으니 개인별 정확한 수강 가능 여부는 관할 고용센터에 확인 부탁드립니다. 관할 고용센터 정보는 고용노동부 대표번호 1350 또는 거주지 근처 지도검색, HRD-Net에서 확인 가능합니다.


* database를 retriever로 선언하기

In [36]:
retriever = database.as_retriever(search_kwargs={'k' : k})

* 모델 선언

In [37]:
chat = ChatOpenAI(model='gpt-3.5-turbo')
qa = RetrievalQA.from_llm(llm=chat, retriever=retriever, return_source_documents=True, )

* 모델 확인

In [38]:
result = qa(query)

In [39]:
print(result["result"])

네, 에이블 스쿨에는 몇 가지 지원 조건이 있습니다. 예를 들어, 교육 시작일 기준으로 34세 이하를 대상으로 하거나, 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 합니다. 또한, AI개발자 Track의 경우에는 기본적인 코딩 역량이 필요합니다. 그 외에도 군 복무 중인 경우에는 교육 시작 전일까지 병역 의무를 마쳐야 하며, 다른 K-DT 과정을 이미 들은 경우에는 일부 제한이 있을 수 있습니다.
